In [1]:
import pandas
import pickle
import itertools
import re
import math
import numpy
from pg import DB
db = DB()
q = db.query("SELECT motif_string FROM motifss order by shuffled_proteins_length")

In [61]:
# db.query("ALTER TABLE motifss ADD COLUMN average_score float;")
# db.query("ALTER TABLE motifss ADD COLUMN shuffled_average_score float;")
# db.query("ALTER TABLE motifss ADD COLUMN tt_average_score float;")
# db.query("ALTER TABLE motifss ADD COLUMN tt_shuffled_average_score float;")

In [ ]:
%%time
i = 0
for motif_t in q.getresult():
    motif = motif_t[0]
    if(db.query("SELECT average_score FROM motifss where motif_string = '" + motif +"' and tt_average_score NOTNULL").ntuples() > 0):
        continue
        
    q_result = db.query("SELECT subgraph,shuffled_subgraph FROM motifss where motif_string = '" + motif + "'").getresult()[0]
    subgraph = q_result[0]
    avg_score = pandas.DataFrame(subgraph).mean().mean()
    ninety_p = numpy.percentile(subgraph, 90)
    for i, row in enumerate(subgraph):
        for j, elem in enumerate(row):
            if(elem < math.floor(ninety_p)):
                subgraph[i][j] = None

    shuffled_subgraph = q_result[1]
    shuffled_avg_score = pandas.DataFrame(shuffled_subgraph).mean().mean()
    ninety_p_shuffled = numpy.percentile(shuffled_subgraph, 90)
    for i, row in enumerate(shuffled_subgraph):
        for j, elem in enumerate(row):
            if(elem < math.floor(ninety_p_shuffled)):
                shuffled_subgraph[i][j] = None
    
    tt_avg_score = pandas.DataFrame(subgraph).mean().mean()
    tt_shuffled_avg_score = pandas.DataFrame(shuffled_subgraph).mean().mean()

#     print(motif, avg_score, shuffled_avg_score)
    do_not_print = db.query("UPDATE motifss SET average_score = " + str(avg_score) + " ,tt_average_score = " + str(tt_avg_score) + ", shuffled_average_score = " +str(shuffled_avg_score) + ",tt_shuffled_average_score = '" + str(tt_shuffled_avg_score) + "'  WHERE motif_string  = '" + motif + "'")
#     print(motif, avg_score,tt_avg_score, shuffled_avg_score, tt_shuffled_avg_score)
    

In [3]:
q_result = db.query("SELECT motif_string FROM motif where proteins_length < 30").getresult()
q_result

[('accccgag',),
 ('accccgat',),
 ('accccgca',),
 ('accccgac',),
 ('accccgcg',),
 ('accccgga',),
 ('accccgta',),
 ('accccggt',),
 ('accgggac',),
 ('acccctag',),
 ('accccgtt',),
 ('acccctat',),
 ('acccctcg',),
 ('acccgaac',),
 ('acccgaaa',),
 ('acccgaca',),
 ('acccgaat',),
 ('acccgacc',),
 ('acccgact',),
 ('acccgaga',),
 ('acccgagt',),
 ('acccgcag',),
 ('acccgcat',),
 ('acccgatg',),
 ('acccgccg',),
 ('acccgcgg',),
 ('acccgcta',),
 ('acccgctc',),
 ('agaccgga',),
 ('acccgcga',),
 ('acccgctg',),
 ('acccggaa',),
 ('acccggac',),
 ('acccggag',),
 ('acccggat',),
 ('acccggcg',),
 ('acccggca',),
 ('acccgggc',),
 ('acccggtc',),
 ('acccgtaa',),
 ('acccggtt',),
 ('acccgtcc',),
 ('acccgtct',),
 ('acccgtgc',),
 ('acccgtta',),
 ('acccgttg',),
 ('acccgttt',),
 ('acccgttc',),
 ('accctacg',),
 ('accctagt',),
 ('accctcgt',),
 ('accgttgt',),
 ('accctcgc',),
 ('accctcgg',),
 ('accctcga',),
 ('acccttcg',),
 ('accgaaac',),
 ('accgaaca',),
 ('accgaaag',),
 ('accgaacg',),
 ('accgaagc',),
 ('accgaaga',),
 ('accga

In [52]:
q_result = db.query("SELECT subgraph,shuffled_subgraph FROM motif where motif_string = 'cgatcgcg'").getresult()[0]
subgraph = q_result[0]
shuffled_subgraph= q_result[1]
ninety_p = numpy.percentile(subgraph, 90)
# print(subgraph)
for i, row in enumerate(subgraph):
    for j, elem in enumerate(row):
        print(elem, ninety_p)
        print(elem < math.floor(ninety_p))
        if(elem < ninety_p):
            subgraph[i][j] = None
# print(subgraph)

-0.0127478914251 -0.0127478914251
False
-0.0127478914251 -0.0127478914251
False


In [24]:
pandas.DataFrame(shuffled_subgraph)

,0
0,-0.012748
1,-0.012748
2,-0.012748
3,-0.012748
4,-0.012748
5,-0.012748
6,-0.012748
7,-0.012748
8,-0.012748
9,-0.012748


In [86]:
pandas.DataFrame(shuffled_subgraph).mean().mean()

0.021063092539264923

In [87]:
df = pandas.DataFrame(shuffled_subgraph)
sum(df.mean())/len(df.mean())

0.02106309253926492

In [94]:
# # ninety = math.floor(len(shuffled_subgraph)*0.9)
# # shuffled_subgraph_df = pandas.DataFrame(shuffled_subgraph)
# # tt_shuffled_subgraph = []
# import numpy
# ninety_p = numpy.percentile(shuffled_subgraph, 90)

for i, row in enumerate(shuffled_subgraph):
    for j, elem in enumerate(row):
        if(elem < ninety_p):
            shuffled_subgraph[i][j] = None
# shuffled_subgraph = for i, row in enumerate(shuffled_subgraph) for j, elem in enumerate(row)

SyntaxError: invalid syntax (<ipython-input-94-6e52def86dab>, line 11)

In [17]:
motif

'cgatcgcg'

In [90]:
pandas.DataFrame(shuffled_subgraph).mean().mean()

0.40720236956011113

In [29]:
foo = db.query("SELECT * FROM motif where motif_string = 'cgatcgcg'").getresult()

In [34]:
foo[0][3]

[[-0.0127478914251], [-0.0127478914251]]

In [47]:
-0.0127478914251 < -0.0127478914251

False